# Algorithm 27: Torsion Angle Loss

The Torsion Angle Loss supervises the predicted backbone and sidechain torsion angles (φ, ψ, ω, χ1-χ4). It uses a sin/cos representation to properly handle the periodic nature of angles.

## Algorithm Pseudocode

![torsionAngleLoss](../imgs/algorithms/torsionAngleLoss.png)

## Source Code Location
- **File**: `AF2-source-code/model/folding.py`
- **Function**: `supervised_chi_loss`
- **Lines**: 480-550

## Overview

### Torsion Angles in Proteins

| Angle | Bond | Description | Typical Range |
|-------|------|-------------|---------------|
| ω (omega) | C-N | Peptide bond | ~180° (trans) |
| φ (phi) | N-Cα | Backbone | -180° to 180° |
| ψ (psi) | Cα-C | Backbone | -180° to 180° |
| χ1-χ4 | Sidechain | Rotameric | Residue-dependent |

### Why Sin/Cos Representation?

1. **Periodicity**: Angles wrap around (180° = -180°)
2. **Smooth Loss**: L2 loss on sin/cos is smooth across the wrap
3. **Normalization**: Unit circle representation has bounded magnitude

In [ ]:
import numpy as np

np.random.seed(42)

## NumPy Implementation

In [ ]:
def angle_to_sincos(angles):
    """
    Convert angles to (sin, cos) representation.
    
    Args:
        angles: Angles in radians [...]
    
    Returns:
        sincos: [..., 2] array with (sin, cos)
    """
    return np.stack([np.sin(angles), np.cos(angles)], axis=-1)


def sincos_to_angle(sincos):
    """
    Convert (sin, cos) back to angle.
    
    Args:
        sincos: [..., 2] array with (sin, cos)
    
    Returns:
        angles: Angles in radians
    """
    return np.arctan2(sincos[..., 0], sincos[..., 1])


def normalize_sincos(sincos):
    """
    Normalize (sin, cos) to unit circle.
    
    Args:
        sincos: [..., 2] array
    
    Returns:
        Normalized sincos on unit circle
    """
    norm = np.sqrt(np.sum(sincos ** 2, axis=-1, keepdims=True) + 1e-8)
    return sincos / norm

In [ ]:
def torsion_angle_loss(pred_angles, gt_angles, angle_mask):
    """
    Torsion Angle Loss - Algorithm 27.
    
    Computes loss between predicted and ground truth torsion angles.
    Uses (sin, cos) representation for proper angular comparison.
    
    Args:
        pred_angles: Predicted angles as (sin, cos) [N_res, 7, 2]
        gt_angles: Ground truth angles as (sin, cos) [N_res, 7, 2]
        angle_mask: Valid angle mask [N_res, 7]
    
    Returns:
        Loss dictionary with individual components
    """
    N_res = pred_angles.shape[0]
    
    print(f"Torsion Angle Loss")
    print(f"="*50)
    print(f"Residues: {N_res}")
    print(f"Angles per residue: 7 (omega, phi, psi, chi1-4)")
    
    # Step 1: Normalize predictions to unit circle
    pred_normalized = normalize_sincos(pred_angles)
    
    print(f"\nStep 1: Normalized predictions")
    pred_norms = np.sqrt(np.sum(pred_normalized ** 2, axis=-1))
    print(f"  Norm range: [{pred_norms.min():.4f}, {pred_norms.max():.4f}]")
    
    # Step 2: Compute squared difference in (sin, cos) space
    # This correctly handles angular periodicity
    diff = pred_normalized - gt_angles
    sq_diff = np.sum(diff ** 2, axis=-1)  # [N_res, 7]
    
    print(f"\nStep 2: Squared difference")
    print(f"  Shape: {sq_diff.shape}")
    print(f"  Range: [{sq_diff.min():.4f}, {sq_diff.max():.4f}]")
    
    # Step 3: Apply mask
    masked_diff = sq_diff * angle_mask
    
    n_valid = np.sum(angle_mask)
    print(f"\nStep 3: Applied mask")
    print(f"  Valid angles: {int(n_valid)}")
    
    # Step 4: Compute mean loss over valid angles
    total_loss = np.sum(masked_diff) / (n_valid + 1e-8)
    
    # Compute per-angle-type losses
    angle_names = ['omega', 'phi', 'psi', 'chi1', 'chi2', 'chi3', 'chi4']
    per_angle_loss = {}
    for i, name in enumerate(angle_names):
        mask_i = angle_mask[:, i]
        n_i = np.sum(mask_i)
        if n_i > 0:
            loss_i = np.sum(masked_diff[:, i]) / n_i
        else:
            loss_i = 0.0
        per_angle_loss[name] = loss_i
    
    print(f"\nStep 4: Per-angle losses")
    for name, loss in per_angle_loss.items():
        print(f"  {name}: {loss:.4f}")
    
    print(f"\nTotal loss: {total_loss:.4f}")
    
    return {
        'total': total_loss,
        'per_angle': per_angle_loss,
        'n_valid': n_valid,
    }

## Test Examples

In [ ]:
# Test 1: Basic functionality
print("Test 1: Basic Functionality")
print("="*60)

N_res = 32

# Ground truth angles (radians)
gt_angles_raw = np.random.randn(N_res, 7) * 0.5
gt_angles = angle_to_sincos(gt_angles_raw)

# Predicted angles with some noise
pred_angles_raw = gt_angles_raw + np.random.randn(N_res, 7) * 0.1
pred_angles = angle_to_sincos(pred_angles_raw)

# Mask: all backbone angles valid, some chi angles masked
angle_mask = np.ones((N_res, 7))
angle_mask[:, 4:] = np.random.choice([0, 1], size=(N_res, 3))  # Random chi mask

loss = torsion_angle_loss(pred_angles, gt_angles, angle_mask)

In [ ]:
# Test 2: Loss vs prediction quality
print("\nTest 2: Loss vs Prediction Quality")
print("="*60)

N_res = 64
gt_angles_raw = np.random.randn(N_res, 7)
gt_angles = angle_to_sincos(gt_angles_raw)
angle_mask = np.ones((N_res, 7))

print(f"Noise level -> Loss:")
for noise_level in [1.0, 0.5, 0.1, 0.05, 0.01, 0.0]:
    pred_noisy = gt_angles_raw + np.random.randn(N_res, 7) * noise_level
    pred = angle_to_sincos(pred_noisy)
    
    # Compute loss silently
    pred_norm = normalize_sincos(pred)
    diff = pred_norm - gt_angles
    sq_diff = np.sum(diff ** 2, axis=-1)
    loss = np.mean(sq_diff)
    
    print(f"  σ={noise_level:.2f}: Loss={loss:.6f}")

In [ ]:
# Test 3: Periodicity handling
print("\nTest 3: Periodicity Handling")
print("="*60)

# Compare angle differences
angle1 = 170 * np.pi / 180  # 170°
angle2 = -170 * np.pi / 180  # -170° (same as 190°)

# Raw angle difference (wrong for periodicity)
raw_diff = angle1 - angle2
print(f"Angles: {np.degrees(angle1):.1f}°, {np.degrees(angle2):.1f}°")
print(f"Raw difference: {np.degrees(raw_diff):.1f}° (WRONG - should be 20°)")

# Sin/cos difference (correct)
sc1 = angle_to_sincos(angle1)
sc2 = angle_to_sincos(angle2)
sincos_diff = np.sqrt(np.sum((sc1 - sc2) ** 2))

# True angular difference
true_diff = 20 * np.pi / 180
expected_sincos_diff = np.sqrt(np.sum((angle_to_sincos(true_diff) - angle_to_sincos(0)) ** 2))

print(f"\nSin/cos L2 difference: {sincos_diff:.4f}")
print(f"Expected for 20° difference: {expected_sincos_diff:.4f}")
print(f"Close: {np.allclose(sincos_diff, expected_sincos_diff, atol=0.1)}")

In [ ]:
# Test 4: Unnormalized predictions
print("\nTest 4: Normalization Effect")
print("="*60)

N_res = 16
gt_angles_raw = np.random.randn(N_res, 7) * 0.5
gt_angles = angle_to_sincos(gt_angles_raw)

# Predictions with various magnitudes
pred_angles_raw = gt_angles_raw.copy()
pred_unnorm = angle_to_sincos(pred_angles_raw) * np.random.uniform(0.5, 2.0, (N_res, 7, 1))

angle_mask = np.ones((N_res, 7))

# Without normalization
diff_unnorm = pred_unnorm - gt_angles
loss_unnorm = np.mean(np.sum(diff_unnorm ** 2, axis=-1))

# With normalization
pred_norm = normalize_sincos(pred_unnorm)
diff_norm = pred_norm - gt_angles
loss_norm = np.mean(np.sum(diff_norm ** 2, axis=-1))

print(f"Without normalization: {loss_unnorm:.4f}")
print(f"With normalization: {loss_norm:.4f}")
print(f"\nNormalization ensures predictions lie on unit circle")

## Verification: Key Properties

In [ ]:
print("Verification: Key Properties")
print("="*60)

N_res = 50
gt_raw = np.random.randn(N_res, 7)
gt = angle_to_sincos(gt_raw)
angle_mask = np.ones((N_res, 7))

# Property 1: Zero loss for identical predictions
pred_identical = gt.copy()
pred_norm = normalize_sincos(pred_identical)
diff = pred_norm - gt
loss_identical = np.mean(np.sum(diff ** 2, axis=-1))
print(f"Property 1 - Zero loss for identical: {loss_identical < 1e-10} (loss={loss_identical:.2e})")

# Property 2: Loss is non-negative
pred_random = angle_to_sincos(np.random.randn(N_res, 7))
pred_norm = normalize_sincos(pred_random)
diff = pred_norm - gt
loss = np.mean(np.sum(diff ** 2, axis=-1))
print(f"Property 2 - Non-negative: {loss >= 0} (loss={loss:.4f})")

# Property 3: Maximum loss is bounded (max 4 for unit vectors)
# Opposite vectors: (sin, cos) vs (-sin, -cos) has diff = (2sin, 2cos), sq = 4
pred_opposite = -gt
diff = pred_opposite - gt
loss_opposite = np.mean(np.sum(diff ** 2, axis=-1))
print(f"Property 3 - Max loss bounded: {loss_opposite <= 4} (loss={loss_opposite:.4f})")

# Property 4: Symmetric in sin/cos
# L2 loss is symmetric
diff_1 = normalize_sincos(pred_random) - gt
diff_2 = gt - normalize_sincos(pred_random)
loss_1 = np.sum(diff_1 ** 2)
loss_2 = np.sum(diff_2 ** 2)
print(f"Property 4 - Symmetric: {np.allclose(loss_1, loss_2)}")

## Source Code Reference

```python
# From AF2-source-code/model/folding.py

def supervised_chi_loss(ret, batch, value):
  """Computes loss for torsion angles.

  Jumper et al. (2021) Suppl. Alg. 27 "torsionAngleLoss"
  """
  chi_mask = batch['chi_mask']
  true_chi = batch['chi_angles']
  pred_angles = ret['sidechains']['angles_sin_cos']
  
  # Normalize predictions to unit circle
  pred_angles = pred_angles / jnp.linalg.norm(
      pred_angles, axis=-1, keepdims=True)
  
  # Compute squared error in (sin, cos) space
  sq_chi_error = jnp.sum(
      squared_difference(pred_angles, true_chi), axis=-1)
  
  # Mean over valid chi angles
  sq_chi_loss = utils.mask_mean(
      mask=chi_mask,
      value=sq_chi_error)
  
  # Also compute norm loss to encourage unit vectors
  angle_norm = jnp.linalg.norm(pred_angles, axis=-1)
  norm_error = jnp.abs(angle_norm - 1.0)
  
  return {
      'chi_loss': sq_chi_loss,
      'norm_loss': utils.mask_mean(chi_mask, norm_error),
  }
```

## Key Insights

1. **Sin/Cos Representation**: Using (sin θ, cos θ) instead of θ directly handles the 360° periodicity naturally.

2. **Normalization**: Predictions are normalized to the unit circle to ensure valid angle representation.

3. **Euclidean Loss**: L2 loss in sin/cos space corresponds to chord distance on the unit circle, which is smooth and differentiable.

4. **Per-Residue Masking**: Chi angles are masked per-residue based on amino acid type (e.g., ALA has no chi angles).

5. **Backbone vs Sidechain**: Backbone angles (φ, ψ, ω) are always present; chi angles depend on amino acid type.

6. **Norm Loss**: An auxiliary loss encourages predictions to have unit norm, improving training stability.